In [1]:
from search_space import distribution_names as df
from search_space import NaturalSearchSpace as N
from search_space import ContinueSearchSpace as R
from search_space import UniversalVariable as x

In [2]:
import inspect
import types

def delegate(f):
    def ff():
        return f()
    return ff

def for_all_methods(decorator):
    def decorate(cls):
        for attr in cls.__dict__: # there's propably a better way to do this
            if callable(getattr(cls, attr)):
                setattr(cls, attr, decorator(getattr(cls, attr)))
        return cls
    return decorate

def __new__(cls, m: float, n: float):
    cls.x_domain = R()
    cls.y_domain = m * cls.x_domain + n
    for n, func in inspect.getmembers(cls):
        cls.__setattr__(cls, n, types.MethodType(delegate(func), cls)) 
    return cls

class LineSearchSpace:
    M = R()
    N = R()
    X = R()

    def __init__(self, m: float = M, n: float = N) -> None:
        self.m, self.n = m, n

    def sampler_lineal_point(self, x: float = X):
        return (x, self.m * x + self.n)

In [3]:
def fun(a, b):
    return a**b
  
# import required modules 
import inspect 
  
# use getargspec() 
print(inspect.getargspec(fun)) 

ArgSpec(args=['a', 'b'], varargs=None, keywords=None, defaults=None)


/var/folders/g_/0lqz2qlx7cx96d901w35swh40000gn/T/ipykernel_6979/2031524247.py:8: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  print(inspect.getargspec(fun))


In [4]:
class Point:
    """Domain Description"""
    x_domain = R()
    y_domain = R()

    """Class Description"""
    def __init__(self, x: float = x_domain, y: float = y_domain) -> None:
        self.x, self.y = x, y

class Line:
    """Domain Description"""
    m_domain = R()
    n_domain = R()
    x_domain = R()

    """Class Description"""
    def __init__(self, m: float = m_domain, n: float = n_domain) -> None:
        self.m, self.n = m, n

    def get_point(self, x: float = x_domain) -> Point:
        return Point(x, self.m * x + self.n)

In [5]:
class B:
    def __init__(self, x) -> None:
        self.x = x


class A:
    B = B(1)

In [6]:
a = A()

In [7]:
a.B.x = 4

In [8]:
e = A()

In [9]:
e.B.x

4

In [13]:
from copy import copy
from ctypes import resize
from typing import Type
from search_space.spaces import SearchSpace, SearchSpaceDomain
import inspect 
  
# use getargspec() 

class ObjectDomain(SearchSpaceDomain):
    def __init__(self, cls) -> None:
        super().__init__()
        self.cls = cls

    #################################################################
    #                                                               #
    #                  Transformations                              #
    #                                                               #
    #################################################################

    @visitor.on("node")
    def transform(self, node, context):
        pass

    @visitor.when(ast.UniversalVariable)
    def transform(self, node, context: SamplerContext):
        raise UnSupportOpError(self, node, "transform")


    #################################################################
    #                                                               #
    #             Class Transformations                             #
    #                                                               #
    #################################################################

    @visitor.when(ast.Index)
    def transform(self, node, context: SamplerContext):
        name = node.index
        ss = self.cls.__dict__[name]
        ss.constraint_list.append(node.ast)

        return self    

    #################################################################
    #                                                               #
    #                  Simple Transform                             #
    #                                                               #
    #################################################################

    @visitor.when(ast.SelfValue)
    def transform(self, node: ast.SelfValue, context: SamplerContext):
        return self

    @visitor.when(ast.NaturalValue)
    def transform(self, node: ast.NaturalValue, context: SamplerContext):
        try:
            return node.other.get_sampler(context=context)[0]
        except AttributeError:
            return node.other


class MetaClassFabricSearchSpace(Type):
    class_decorated = False

    def __call__(cls, *args, **kwargs):
        try:
            logs_name = kwargs["log_name"]
        except:
            logs_name = None

        if not cls.class_decorated:
            def decorator(func):
                def f(*args, **kwargs):
                    _self = args[0]

                    try:
                        context = kwargs["__context__"]
                        fabric = kwargs["__fabric__"]
                    except KeyError:
                        context = _self.__context__
                        fabric = _self.__fabric__

                    new_kwargs = {} 
                    func_data = inspect.getfullargspec(func)
                    args_len = len(func_data.args)
                    defaults_len = len(func_data.defaults)
                    for index in range(defaults_len):
                        
                        ss = func_data.defaults[defaults_len - 1 - index]
                        
                        if not isinstance(ss, SearchSpace):
                            try:
                                value = args[args_len - 1 - index]
                                new_kwargs[func_data.args[args_len - 1 - index]]  = value
                            except IndexError:
                                new_kwargs[func_data.args[args_len - 1 - index]]  = ss

                            continue

                        for key, item in _self.__class__.__dict__.items():
                            if ss.is_equal(item):  # check instance
                               ss = fabric.__dict__[key].get_sampler(context)[0]
                               break
                            
                        try:
                            # check rules 
                            value = args[args_len - 1 - index]
                            context.registry_sampler(ss, value)

                        except IndexError:
                            value = ss.get_sampler(context) 

                        new_kwargs[func_data.args[args_len - 1 - index]]  = value
                    args = args[0: args_len - defaults_len]
                    return func(*args, **new_kwargs)
                return f

            for attr in cls.__dict__:  # there's propably a better way to do this
                if callable(getattr(cls, attr)):
                    setattr(cls, attr, decorator(getattr(cls, attr)))

        oss = FabricSearchSpace(cls, (args, kwargs), log_name=logs_name)
        for key, item in cls.__dict__.items():
            if isinstance(item, SearchSpace):
                oss.__dict__[key] = copy(item)

        return oss


class FabricSearchSpace(SearchSpace):
    def __init__(self, cls, initial_data, log_name=None) -> None:
        super().__init__(None, None, log_name)
        self.cls = cls
        self.initial_domain = initial_data

    def _get_random_value(self, domain, context):
        args, kw = self.initial_domain
        result = type.__call__(self.cls, *args, **kw, __context__ = context, __fabric__ = self)
        result.__context__ = context
        result.__fabric__ = self
        return result
    
    def _create_domain(self, domain) -> SearchSpaceDomain:
        return ObjectDomain(self)



class Line(metaclass = MetaClassFabricSearchSpace):
    """Domain Description"""
    m_domain = R()
    n_domain = R()
    x_domain = R()

    """Class Description"""
    def __init__(self, m: float = m_domain, n: float = n_domain) -> None:
        self.m, self.n = m, n

    def get_point(self, x: float = x_domain) :
        return (x, self.m * x + self.n)

In [14]:
a= Line()

In [15]:
a.get_sampler()

(<__main__.Line at 0x10834e5e0>,
 <search_space.context_manager.sampler_context.SamplerContext at 0x1088effd0>)